In [16]:
from scipy.io import arff
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

from sklearn.model_selection import train_test_split

In [46]:
dataset = arff.loadarff('../data/weather.arff')
df = pd.DataFrame(dataset[0])

STR_COLUMNS = ['outlook', 'windy', 'play']

for column in STR_COLUMNS:
    df[column] = df[column].str.decode('utf-8')
    
df

,outlook,temperature,humidity,windy,play
0,sunny,85.0,85.0,FALSE,no
1,sunny,80.0,90.0,TRUE,no
2,overcast,83.0,86.0,FALSE,yes
3,rainy,70.0,96.0,FALSE,yes
4,rainy,68.0,80.0,FALSE,yes
5,rainy,65.0,70.0,TRUE,no
6,overcast,64.0,65.0,TRUE,yes
7,sunny,72.0,95.0,FALSE,no
8,sunny,69.0,70.0,FALSE,yes
9,rainy,75.0,80.0,FALSE,yes


In [47]:
df_encoded = pd.get_dummies(df, columns=['outlook', 'windy'])
df_encoded = pd.get_dummies(df_encoded, columns=['play'], drop_first=True)
df_encoded.temperature = df_encoded.temperature * 0.01
df_encoded.humidity = df_encoded.humidity * 0.01
df_encoded

,temperature,humidity,outlook_overcast,outlook_rainy,outlook_sunny,windy_FALSE,windy_TRUE,play_yes
0,0.85,0.85,0,0,1,1,0,0
1,0.80,0.90,0,0,1,0,1,0
2,0.83,0.86,1,0,0,1,0,1
3,0.70,0.96,0,1,0,1,0,1
4,0.68,0.80,0,1,0,1,0,1
5,0.65,0.70,0,1,0,0,1,0
6,0.64,0.65,1,0,0,0,1,1
7,0.72,0.95,0,0,1,1,0,0
8,0.69,0.70,0,0,1,1,0,1
9,0.75,0.80,0,1,0,1,0,1


In [48]:
X = df_encoded.iloc[:, :7].values
X

array([[0.85, 0.85, 0.  , 0.  , 1.  , 1.  , 0.  ],
       [0.8 , 0.9 , 0.  , 0.  , 1.  , 0.  , 1.  ],
       [0.83, 0.86, 1.  , 0.  , 0.  , 1.  , 0.  ],
       [0.7 , 0.96, 0.  , 1.  , 0.  , 1.  , 0.  ],
       [0.68, 0.8 , 0.  , 1.  , 0.  , 1.  , 0.  ],
       [0.65, 0.7 , 0.  , 1.  , 0.  , 0.  , 1.  ],
       [0.64, 0.65, 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.72, 0.95, 0.  , 0.  , 1.  , 1.  , 0.  ],
       [0.69, 0.7 , 0.  , 0.  , 1.  , 1.  , 0.  ],
       [0.75, 0.8 , 0.  , 1.  , 0.  , 1.  , 0.  ],
       [0.75, 0.7 , 0.  , 0.  , 1.  , 0.  , 1.  ],
       [0.72, 0.9 , 1.  , 0.  , 0.  , 0.  , 1.  ],
       [0.81, 0.75, 1.  , 0.  , 0.  , 1.  , 0.  ],
       [0.71, 0.91, 0.  , 1.  , 0.  , 0.  , 1.  ]])

In [49]:
y = df_encoded.iloc[:, 7:].values
y

array([[0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0]], dtype=uint8)

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=17779)

In [60]:
print(len(X_train))

12


In [62]:
print(len(X_test))

2


In [58]:
model = Sequential()
model.add(Dense(16,
                activation='sigmoid',
                input_shape=(7,)))
model.add(Dense(32,
                activation='sigmoid'))
model.add(Dense(16,
                activation='sigmoid'))
model.add(Dense(8,
                activation='sigmoid'))
model.add(Dense(1,
                activation='sigmoid'))
model.summary()

sgd = optimizers.SGD(momentum=0.0)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=4,
                    epochs=1000)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 16)                128       
_________________________________________________________________
dense_50 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_51 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_52 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_53 (Dense)             (None, 1)                 9         
Total params: 1,345
Trainable params: 1,345
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
12/12 [==============================] - 0s 36ms/step - loss: 0.7571 - acc: 0.3333
Epoch 2/1000
12/12 [========================

Epoch 75/1000
12/12 [==============================] - 0s 756us/step - loss: 0.6382 - acc: 0.6667
Epoch 76/1000
12/12 [==============================] - 0s 664us/step - loss: 0.6381 - acc: 0.6667
Epoch 77/1000
12/12 [==============================] - 0s 936us/step - loss: 0.6381 - acc: 0.6667
Epoch 78/1000
12/12 [==============================] - 0s 620us/step - loss: 0.6380 - acc: 0.6667
Epoch 79/1000
12/12 [==============================] - 0s 553us/step - loss: 0.6379 - acc: 0.6667
Epoch 80/1000
12/12 [==============================] - 0s 936us/step - loss: 0.6397 - acc: 0.6667
Epoch 81/1000
12/12 [==============================] - 0s 625us/step - loss: 0.6378 - acc: 0.6667
Epoch 82/1000
12/12 [==============================] - 0s 884us/step - loss: 0.6387 - acc: 0.6667
Epoch 83/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6396 - acc: 0.6667
Epoch 84/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6377 - acc: 0.6667
Epoch 85/1000
12/12 [===

12/12 [==============================] - 0s 556us/step - loss: 0.6369 - acc: 0.6667
Epoch 241/1000
12/12 [==============================] - 0s 715us/step - loss: 0.6379 - acc: 0.6667
Epoch 242/1000
12/12 [==============================] - 0s 814us/step - loss: 0.6379 - acc: 0.6667
Epoch 243/1000
12/12 [==============================] - 0s 783us/step - loss: 0.6379 - acc: 0.6667
Epoch 244/1000
12/12 [==============================] - 0s 680us/step - loss: 0.6379 - acc: 0.6667
Epoch 245/1000
12/12 [==============================] - 0s 716us/step - loss: 0.6369 - acc: 0.6667
Epoch 246/1000
12/12 [==============================] - 0s 654us/step - loss: 0.6369 - acc: 0.6667
Epoch 247/1000
12/12 [==============================] - 0s 899us/step - loss: 0.6369 - acc: 0.6667
Epoch 248/1000
12/12 [==============================] - 0s 600us/step - loss: 0.6379 - acc: 0.6667
Epoch 249/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6379 - acc: 0.6667
Epoch 250/1000
12/12 [=====

12/12 [==============================] - 0s 852us/step - loss: 0.6369 - acc: 0.6667
Epoch 407/1000
12/12 [==============================] - 0s 819us/step - loss: 0.6378 - acc: 0.6667
Epoch 408/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6378 - acc: 0.6667
Epoch 409/1000
12/12 [==============================] - 0s 913us/step - loss: 0.6388 - acc: 0.6667
Epoch 410/1000
12/12 [==============================] - 0s 973us/step - loss: 0.6369 - acc: 0.6667
Epoch 411/1000
12/12 [==============================] - 0s 489us/step - loss: 0.6369 - acc: 0.6667
Epoch 412/1000
12/12 [==============================] - 0s 762us/step - loss: 0.6369 - acc: 0.6667
Epoch 413/1000
12/12 [==============================] - 0s 500us/step - loss: 0.6369 - acc: 0.6667
Epoch 414/1000
12/12 [==============================] - 0s 854us/step - loss: 0.6369 - acc: 0.6667
Epoch 415/1000
12/12 [==============================] - 0s 663us/step - loss: 0.6369 - acc: 0.6667
Epoch 416/1000
12/12 [=====

12/12 [==============================] - 0s 945us/step - loss: 0.6369 - acc: 0.6667
Epoch 572/1000
12/12 [==============================] - 0s 731us/step - loss: 0.6369 - acc: 0.6667
Epoch 573/1000
12/12 [==============================] - 0s 628us/step - loss: 0.6387 - acc: 0.6667
Epoch 574/1000
12/12 [==============================] - 0s 923us/step - loss: 0.6369 - acc: 0.6667
Epoch 575/1000
12/12 [==============================] - 0s 671us/step - loss: 0.6369 - acc: 0.6667
Epoch 576/1000
12/12 [==============================] - 0s 790us/step - loss: 0.6369 - acc: 0.6667
Epoch 577/1000
12/12 [==============================] - 0s 811us/step - loss: 0.6378 - acc: 0.6667
Epoch 578/1000
12/12 [==============================] - 0s 500us/step - loss: 0.6369 - acc: 0.6667
Epoch 579/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6369 - acc: 0.6667
Epoch 580/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6378 - acc: 0.6667
Epoch 581/1000
12/12 [=======

Epoch 654/1000
12/12 [==============================] - 0s 477us/step - loss: 0.6369 - acc: 0.6667
Epoch 655/1000
12/12 [==============================] - 0s 837us/step - loss: 0.6369 - acc: 0.6667
Epoch 656/1000
12/12 [==============================] - 0s 878us/step - loss: 0.6369 - acc: 0.6667
Epoch 657/1000
12/12 [==============================] - 0s 668us/step - loss: 0.6369 - acc: 0.6667
Epoch 658/1000
12/12 [==============================] - 0s 841us/step - loss: 0.6369 - acc: 0.6667
Epoch 659/1000
12/12 [==============================] - 0s 765us/step - loss: 0.6378 - acc: 0.6667
Epoch 660/1000
12/12 [==============================] - 0s 589us/step - loss: 0.6387 - acc: 0.6667
Epoch 661/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6369 - acc: 0.6667
Epoch 662/1000
12/12 [==============================] - 0s 789us/step - loss: 0.6369 - acc: 0.6667
Epoch 663/1000
12/12 [==============================] - 0s 644us/step - loss: 0.6369 - acc: 0.6667
Epoch 664/10

12/12 [==============================] - 0s 883us/step - loss: 0.6387 - acc: 0.6667
Epoch 738/1000
12/12 [==============================] - 0s 810us/step - loss: 0.6369 - acc: 0.6667
Epoch 739/1000
12/12 [==============================] - 0s 872us/step - loss: 0.6369 - acc: 0.6667
Epoch 740/1000
12/12 [==============================] - 0s 676us/step - loss: 0.6369 - acc: 0.6667
Epoch 741/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6378 - acc: 0.6667
Epoch 742/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6387 - acc: 0.6667
Epoch 743/1000
12/12 [==============================] - 0s 944us/step - loss: 0.6369 - acc: 0.6667
Epoch 744/1000
12/12 [==============================] - 0s 660us/step - loss: 0.6369 - acc: 0.6667
Epoch 745/1000
12/12 [==============================] - 0s 897us/step - loss: 0.6369 - acc: 0.6667
Epoch 746/1000
12/12 [==============================] - 0s 832us/step - loss: 0.6369 - acc: 0.6667
Epoch 747/1000
12/12 [=======

12/12 [==============================] - 0s 594us/step - loss: 0.6369 - acc: 0.6667
Epoch 821/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6387 - acc: 0.6667
Epoch 822/1000
12/12 [==============================] - 0s 759us/step - loss: 0.6387 - acc: 0.6667
Epoch 823/1000
12/12 [==============================] - 0s 683us/step - loss: 0.6369 - acc: 0.6667
Epoch 824/1000
12/12 [==============================] - 0s 672us/step - loss: 0.6378 - acc: 0.6667
Epoch 825/1000
12/12 [==============================] - 0s 1ms/step - loss: 0.6378 - acc: 0.6667
Epoch 826/1000
12/12 [==============================] - 0s 657us/step - loss: 0.6387 - acc: 0.6667
Epoch 827/1000
12/12 [==============================] - 0s 593us/step - loss: 0.6369 - acc: 0.6667
Epoch 828/1000
12/12 [==============================] - 0s 703us/step - loss: 0.6387 - acc: 0.6667
Epoch 829/1000
12/12 [==============================] - 0s 536us/step - loss: 0.6369 - acc: 0.6667
Epoch 830/1000
12/12 [=======

12/12 [==============================] - 0s 618us/step - loss: 0.6369 - acc: 0.6667
Epoch 904/1000
12/12 [==============================] - 0s 927us/step - loss: 0.6369 - acc: 0.6667
Epoch 905/1000
12/12 [==============================] - 0s 585us/step - loss: 0.6387 - acc: 0.6667
Epoch 906/1000
12/12 [==============================] - 0s 885us/step - loss: 0.6369 - acc: 0.6667
Epoch 907/1000
12/12 [==============================] - 0s 946us/step - loss: 0.6368 - acc: 0.6667
Epoch 908/1000
12/12 [==============================] - 0s 985us/step - loss: 0.6387 - acc: 0.6667
Epoch 909/1000
12/12 [==============================] - 0s 758us/step - loss: 0.6387 - acc: 0.6667
Epoch 910/1000
12/12 [==============================] - 0s 643us/step - loss: 0.6368 - acc: 0.6667
Epoch 911/1000
12/12 [==============================] - 0s 492us/step - loss: 0.6368 - acc: 0.6667
Epoch 912/1000
12/12 [==============================] - 0s 777us/step - loss: 0.6368 - acc: 0.6667
Epoch 913/1000
12/12 [===

In [24]:
y_pred = model.predict(X_test)

In [27]:
from sklearn.metrics import recall_score

y_pred = np.round(y_pred)
print(y_pred)
print(recall_score(y_pred, y_test))

[[1.]
 [1.]]
0.5


In [23]:
# model.save('../model/keras-adam.h5')
# model.save('../model/keras-sgd.h5')

In [7]:
model.predict(X_test)

NameError: name 'X_test' is not defined

In [3]:
df = pd.read_csv('../data/train.csv')
df.fillna(df.median(), inplace=True)
X = df.drop(columns=['Name', 'Ticket', 'Cabin', 'SibSp', 'Parch', 'Embarked'])
y = X['Survived']
X = X.drop(columns=['Survived'])
X = pd.get_dummies(X, columns=['Sex'])

In [51]:
model = Sequential()
model.add(Dense(16,
                activation='sigmoid',
                input_shape=(6,)))
model.add(Dense(32,
                activation='sigmoid'))
model.add(Dense(16,
                activation='sigmoid'))
model.add(Dense(8,
                activation='sigmoid'))
model.add(Dense(1,
                activation='sigmoid'))

sgd = optimizers.SGD(lr=0.01, momentum=0.001)
model.compile(optimizer=sgd,
              loss='mean_squared_error',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 16)                112       
_________________________________________________________________
dense_20 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_21 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_22 (Dense)             (None, 8)                 136       
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 9         
Total params: 1,329
Trainable params: 1,329
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.fit(X, y,
          batch_size=1,
          epochs=1000,
          verbose=1)

ValueError: Error when checking input: expected dense_14_input to have shape (6,) but got array with shape (7,)

In [ ]:
model.predict(X_)